In [11]:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
# Leer el archivo CSV
df = pd.read_csv('D:/MCDAustral/Text Mining/TextMining/RECLAMOS 4.csv', encoding='latin-1')
df

,problema_id,descripcion,alias_8_nombre3,problema_n,obsitem,Concaobsitem
0,70e03aaf-6cfb-42e8-92ae-fcb46ff479dd,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,B2 - PÃ©rdidas de gas sin piezas daÃ±adas,HAY UNA PERILLA DE LAS HORNALLA QUE NO SE PUED...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
1,e5be47a0-c506-4a79-87e9-2a45f06d496c,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,G - Perilla no gira / gira mal,HAY UNA PERILLA DE LAS HORNALLA QUE NO SE PUED...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
2,2b2af341-20c5-4607-baa0-01972617647e,TERMO ELECTRICO EL-55,ELE - Termo elÃ©ctrico,D - No enciende,DESPUÃS DE QUE UNA PERSONA SE DUCHO NO VOLVIÃ...,"TERMO ELECTRICO EL-55 ELE - Termo elÃ©ctrico ""..."
3,ab0ed03a-8aab-4317-9aad-fcc06dd41520,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,D2 - Hornalla Mal funcionamiento,LAS HORNALLAS TARDAN MUCHÃSIMO TIEMPO EN PREN...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
4,29ce3db5-f678-4f14-b86e-90c8265daad5,COCINA CANDOR S2 (GN) (U.),GN - Gas Natural,C2 - Horno Mal funcionamiento,EL QUEMADOR DEL HORNO NO MANTIENE LA LLAMA ENC...,"COCINA CANDOR S2 (GN) (U.) GN - Gas Natural "" ..."
...,...,...,...,...,...,...
15175,5e810261-6a73-470d-b900-5018f9f83927,COCINA MASTER BLACK MULTIGAS (U.),MG - Multi Gas,D3 - Hornalla Se Apaga,LAS HORNALLAS FALLAS AL QUERER ENCENDERLAS,COCINA MASTER BLACK MULTIGAS (U.) MG - Multi G...
15176,29ce3db5-f678-4f14-b86e-90c8265daad5,COCINA MASTER BLACK MULTIGAS (U.),MG - Multi Gas,C2 - Horno Mal funcionamiento,"CUANDO PRENDO LAS HORNALLAS, SE APAGA EL HORNO...",COCINA MASTER BLACK MULTIGAS (U.) MG - Multi G...
15177,8c5fec8e-37bc-46ac-a031-717476ef2f6f,COCINA PALACE CRISTAL LX S2 (GN) (U.),GN - Gas Natural,E - Puerta de horno - No cierra / Se cae,SE ROMPIÃ LA BISAGRA DE LA PUERTA DEL HORNO,COCINA PALACE CRISTAL LX S2 (GN) (U.) GN - Gas...
15178,5e810261-6a73-470d-b900-5018f9f83927,COCINA PALACE CRISTAL LX S2 (GL) (U.),GE - Gas Envasado,D3 - Hornalla Se Apaga,- PROBLEMAS CON EL HORNO.\n \n - EL HORNO PERM...,COCINA PALACE CRISTAL LX S2 (GL) (U.) GE - Gas...


In [28]:
# Obtener las columnas de diagnósticos y códigos AIS
Oitems = df['Concaobsitem'].tolist()
ids = df['problema_id'].tolist()

In [29]:
#Se almacenan los elementos de Oitems como cadenas de caracteres en descripciones
descripciones = [str(desc) for desc in Oitems]

In [30]:
# Codificar los códigos AIS como etiquetas numéricas
label_encoder = LabelEncoder()
codigos_AIS_numericos = label_encoder.fit_transform(ids)

In [31]:
# Función para realizar stemming en una descripción
stemmer = SnowballStemmer('spanish')
def stem_descripcion(descripcion):
    words = nltk.word_tokenize(descripcion)
    words = [stemmer.stem(word) for word in words if word not in stopwords.words('spanish')]
    return ' '.join(words)

In [32]:
# Aplicar stemming a las descripciones
descripciones_stemmed = [stem_descripcion(desc) for desc in descripciones]

In [33]:
# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(descripciones_stemmed, codigos_AIS_numericos, test_size=0.20, random_state=42)

In [34]:
# Creación del pipeline para el clasificador basado en texto
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC())
])

In [35]:
# Entrenamiento del modelo
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SVC())])

In [36]:
# Evaluación del modelo
y_pred = pipeline.predict(X_test)

In [37]:
# Obtener las clases únicas de los datos de prueba y entrenamiento
unique_classes = np.unique(np.concatenate((y_train, y_test)))

In [38]:
# Crear un diccionario para mapear las etiquetas numéricas a los nombres de las clases originales
class_names_dict = {class_num: class_name for class_num, class_name in zip(unique_classes, label_encoder.inverse_transform(unique_classes))}

In [39]:
# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred, labels=unique_classes, target_names=class_names_dict.values(), zero_division=1))

                                      precision    recall  f1-score   support

0329f614-7eaf-4230-b10a-6a11127a1ad6       0.76      0.52      0.62        75
0713deef-4b21-477e-93e9-bbbfb3c1b1c4       0.82      0.26      0.40        34
1ed1a93d-335b-4adb-a565-5929499e8613       1.00      0.00      0.00         1
208f4777-71e7-4897-b746-71e33d105bcf       1.00      0.15      0.27        26
29ce3db5-f678-4f14-b86e-90c8265daad5       0.62      0.69      0.65       139
2b2af341-20c5-4607-baa0-01972617647e       0.63      0.46      0.53       251
2d04c68a-bbb7-4ef9-b293-49bd5c81455d       1.00      0.08      0.14        13
4ab62879-fa8e-4609-8b99-ac8dbeea466c       1.00      0.00      0.00         1
553f6bde-32ee-4099-a99a-15a4ff7729f4       0.52      0.41      0.46       140
5e810261-6a73-470d-b900-5018f9f83927       0.58      0.58      0.58       141
7094cb00-a60c-4963-88b5-ab4335f9c0ee       0.40      0.17      0.24        12
70e03aaf-6cfb-42e8-92ae-fcb46ff479dd       0.33      0.11      

In [51]:
import openpyxl



In [52]:
# Ejemplo de predicción para diagnósticos nuevos
#df = pd.read_csv('D:/MCDAustral/Text Mining/TextMining/RECLAMOS 4.csv', encoding='latin-1')

archivo_diagnosticos_csv = 'D:/MCDAustral/Text Mining/TextMining/testesco.xlsx'  # Cambia la ruta al archivo CSV
df_diagnosticos = pd.read_excel(archivo_diagnosticos_csv)
df_diagnosticos

,obsitem
0,ELE - Termo eléctrico 3150 - Cañerías/Otros ar...
1,GE - Gas Envasado C2 - Horno Mal funcionamient...
2,GN - Gas Natural 1010 - Instalación no realiza...
3,MG - Multi Gas D6 - Otra Hornalla Se Apaga
4,GAS - Termo a gas 2040 - Tiraje incorrecto cte...
5,ELE - Termo eléctrico E - El agua sale con tem...


In [53]:
# Filtrar las clases que existen en el conjunto de entrenamiento
clases_validas_prueba = label_encoder.classes_
df_diagnosticos_filtrados = df_diagnosticos[df_diagnosticos['obsitem'].isin(clases_validas_prueba)]

In [54]:
df_diagnosticos_filtrados

,obsitem


In [55]:
# Codificar las clases en los datos de prueba usando el mismo LabelEncoder
codigos_AIS_numericos_prueba = label_encoder.transform(df_diagnosticos_filtrados['obsitem'])

In [56]:
resultados = []
for diagnostico in df_diagnosticos_filtrados['obsitem']:
    diagnostico_stemmed = stem_descripcion(diagnostico.strip())
    predicciones = pipeline.predict([diagnostico_stemmed])
    
    clase_predicha = label_encoder.inverse_transform(predicciones)[0]
    
    resultados.append({
        'Descripción': diagnostico.strip(),
        'Código AIS predicho': clase_predicha
    })

In [58]:
resultados_df = pd.DataFrame(resultados)

resultados_csv = 'D:/MCDAustral/Text Mining/TextMining/testresultadoSVC2.csv'
resultados_df.to_csv(resultados_csv, index=False, encoding='latin-1')

print("Resultados guardados en:", resultados_csv)

Resultados guardados en: D:/MCDAustral/Text Mining/TextMining/testresultadoSVC2.csv
